# Cài thư viện cho chương trình 

In [18]:
import pandas as pd

# Lấy dữ liệu 
- Lấy dữ liệu từ thư mục raw.csv

In [19]:
data = pd.read_csv('./raw_data.csv')
data.head()

,Name,Pulisher,Price,Net Revenue,Platform,Genres
0,Grand Theft Auto V,Rockstar Games,$29.99,~$700 million,Windows,"Action,Adventure"
1,Cyberpunk 2077,CD PROJEKT RED,$59.99,~$490 million,Windows,RPG
2,Rust,Facepunch Studios,$39.99,~$490 million,Windows Mac,"Action,Adventure,Indie,MassivelyMultiplayer,RPG"
3,The Witcher® 3: Wild Hunt,CD PROJEKT RED,$39.99,~$430 million,Windows,RPG
4,Tom Clancy's Rainbow Six® Siege,Ubisoft,$19.99,~$340 million,Windows,Action


In [20]:
data.shape

(1427, 6)

# Xử lý dữ liệu

Kiểm tra có dữ liệu null hay không

In [21]:
data.isnull().sum()

Name            0
Pulisher        0
Price           0
Net Revenue     0
Platform       30
Genres          2
dtype: int64

Bởi vì dữ liệu trống ít nên sẽ bỏ đi 

In [22]:
data = data.dropna()
data.isnull().sum()

Name           0
Pulisher       0
Price          0
Net Revenue    0
Platform       0
Genres         0
dtype: int64

Lọc ký tự không phải số ra khỏi trường Net Revenue

In [23]:

data['Net Revenue'] = data['Net Revenue'].map(lambda x: x.lstrip('~$').rstrip(''))
data['Net Revenue'] = data['Net Revenue'].map(lambda x: x.replace(' million',''))
print(data["Net Revenue"])

0       700
1       490
2       490
3       430
4       340
       ... 
1422      1
1423      1
1424      1
1425      1
1426      1
Name: Net Revenue, Length: 1395, dtype: object


Lọc ký tự không phải số ra khỏi trường Price

In [24]:
data['Price'] = data['Price'].map(lambda x: x.lstrip('$').rstrip(''))
print(data["Price"])

0       29.99
1       59.99
2       39.99
3       39.99
4       19.99
        ...  
1422     9.99
1423    24.99
1424    19.99
1425    24.99
1426    34.99
Name: Price, Length: 1395, dtype: object


Vẫn còn một vài cell chưa được xử lý

In [25]:
data.loc[354]

Name           Saints Row: The Third
Pulisher                 Deep Silver
Price                          9.99$
Net Revenue                      8.6
Platform               Windows Linux
Genres                        Action
Name: 354, dtype: object

drop các row mà biến Price vẫn còn ký tự không phải số

In [26]:
data = data[pd.to_numeric(data['Price'], errors='coerce').notnull()]

Một vài trường trong dữ liệu có dạng categorical.Mô hình linear-regression trong thư viện này chỉ xử lý với dữ liệu số thực do đó chúng ta cần chuyển các thuộc tính trên về dạng số thực tương ứng. Sử dụng  __skearn.preprocessing__ để chuyển dữ liệu dạng categorical về số thực

In [27]:
data.head()

,Name,Pulisher,Price,Net Revenue,Platform,Genres
0,Grand Theft Auto V,Rockstar Games,29.99,700,Windows,"Action,Adventure"
1,Cyberpunk 2077,CD PROJEKT RED,59.99,490,Windows,RPG
2,Rust,Facepunch Studios,39.99,490,Windows Mac,"Action,Adventure,Indie,MassivelyMultiplayer,RPG"
3,The Witcher® 3: Wild Hunt,CD PROJEKT RED,39.99,430,Windows,RPG
4,Tom Clancy's Rainbow Six® Siege,Ubisoft,19.99,340,Windows,Action


In [28]:
data_clean = data[['Name', 'Pulisher', 'Price', 'Net Revenue', 'Platform', 'Genres']].values
from sklearn import preprocessing
le_pub = preprocessing.LabelEncoder()
le_pub.fit(data.Pulisher.unique().tolist())
data_clean[:,1] = le_pub.transform(data_clean[:,1]) 


le_Genres = preprocessing.LabelEncoder()
le_Genres.fit(data.Genres.unique().tolist())
data_clean[:,5] = le_Genres.transform(data_clean[:,5])


le_Platform = preprocessing.LabelEncoder()
le_Platform.fit(data.Platform.unique().tolist())
data_clean[:,4]  = le_Platform.transform(data_clean[:,4])

clean_df = pd.DataFrame(data_clean, columns=['Name', 'Pulisher', 'Price', 'Net Revenue', 'Platform', 'Genres'])
view_df = clean_df.copy()
view_df["Platform_text"] = data["Platform"]
view_df["Genres_text"] = data["Genres"]
view_df["Pulisher_text"] = data["Pulisher"]
print(clean_df.head())

                              Name Pulisher  Price Net Revenue Platform Genres
0               Grand Theft Auto V      422  29.99         700        0      1
1                   Cyberpunk 2077       91  59.99         490        0    207
2                             Rust      188  39.99         490        3     36
3        The Witcher® 3: Wild Hunt       91  39.99         430        0    207
4  Tom Clancy's Rainbow Six® Siege      549  19.99         340        0      0


# Chuyển dữ liệu clean sang file csv

In [29]:

view_df.to_csv("view.csv", index=False)
clean_df.to_csv("clean_data.csv", index=False)